In [ ]:
# Import stanza
import stanza
#stanza.install_corenlp(dir=corenlp_dir)

# Setting StanfordCoreNLP
import os
import stanfordnlp
import pandas as pd
from tqdm import tqdm
corenlp_dir = './corenlp'
os.environ["CORENLP_HOME"] = corenlp_dir
#from stanza.server import CoreNLPClient
from stanfordnlp.server import CoreNLPClient

# Setting AllenNLP
from allennlp.predictors.predictor import Predictor
from allennlp_models.coref.predictors.coref import CorefPredictor
from allennlp_models.coref.dataset_readers.conll import ConllCorefReader
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

# Initialise CoreNLPClient
nlp = CoreNLPClient(annotators=['kbp'], memory='4G', endpoint='http://localhost:9001', output_format='json')

# Input Text
text = "Huawei Technologies Co., Ltd.is a Chinese multinational technology company headquartered in Shenzhen, Guangdong. It designs, develops, and sells telecommunications equipment and consumer electronics (mainly smartphones). The company was founded in 1987 by Ren Zhengfei, a former Deputy Regimental Chief in the People's Liberation Army. Huawei has deployed its products and services in more than 170 countries. It overtook Ericsson in 2012 as the largest telecommunications equipment manufacturer in the world, and overtook Apple in 2018 as the second-largest manufacturer of smartphones in the world, behind Samsung Electronics. In December 2019, Huawei reported that its annual revenue had risen to $121.72 billion US dollars in 2019. Although successful internationally, Huawei has faced difficulties in some markets, due to claims of undue state support, links to the People's Liberation Army, and cybersecurity concerns—primarily from the United States government—that Huawei's infrastructure equipment may enable surveillance by the Chinese government. With the development of 5G wireless networks, there have been calls from the U.S. and its allies to not do any kind of business with Huawei or other Chinese telecommunications companies such as ZTE. Huawei has argued that its products posed no greater cybersecurity risk than those of any other vendor and that there is no evidence of the U.S. espionage claims. Questions regarding Huawei's ownership and control as well as concerns regarding the extent of state support also remain. Huawei has also been accused of assisting in the surveillance and mass detention of Uyghurs in the Xinjiang re-education camps, which have resulted in sanctions by the United States Department of State. In the midst of an ongoing trade war between China and the United States, Huawei was restricted from doing commerce with U.S. companies due to alleged previous willful violations of U.S. sanctions against Iran. On 29 June 2019, U.S. President Donald Trump reached an agreement to resume trade talks with China and announced that he would ease the aforementioned sanctions on Huawei. Huawei cut 600 jobs at its Santa Clara research center in June, and in December 2019 founder Ren Zhengfei said it was moving the center to Canada because the restrictions would block them from interacting with US employees. In July 2020, Huawei surpassed Samsung and Apple to become the leading smartphone mobile brand in the world for the first time primarily due to a drop in Samsung's global sales in the second quarter of 2020 owing to the impact of the COVID-19 pandemic."

# Resolve the Text (Replace pronounces into references)
allennlp_resolved = predictor.coref_resolved(text)

# Annotate relations between triples
annotated = nlp.annotate(allennlp_resolved)

# Extract the annotated texts
annotated_sentences = annotated['sentences']
rows = []
for s in tqdm(range(len(annotated_sentences))):
    for r in range(s):
        try:
            subject = annotated_sentences[s]['kbp'][r]['subject']
            subjectSpan = annotated_sentences[s]['kbp'][r]['subjectSpan']
            relation = annotated_sentences[s]['kbp'][r]['relation']
            objects = annotated_sentences[s]['kbp'][r]['object']
            objectSpan = annotated_sentences[s]['kbp'][r]['objectSpan']
            row = list([subject, subjectSpan, relation, objects, objectSpan])
            rows.append(row)
        except:
            pass
    
# View output in a dataframe
relation_df = pd.DataFrame(rows, columns = ['Subject', 'SubjectSpan', 'Relation', 'Object', 'ObjectSpan'])
relation_df

Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c7154e4a2b6d46cb.props -preload kbp


100%|██████████| 15/15 [00:00<00:00, 10674.34it/s]


In [69]:
relation_df

,Subject,SubjectSpan,Relation,Object,ObjectSpan
0,Ren Zhengfei,"[10, 12]",per:employee_or_member_of,People 's Liberation Army,"[20, 24]"
1,People 's Liberation Army,"[20, 24]",org:top_members_employees,Ren Zhengfei,"[10, 12]"
2,Donald Trump,"[7, 9]",per:title,President,"[6, 7]"
3,Donald Trump,"[7, 9]",per:countries_of_residence,U.S.,"[5, 6]"
4,Ren Zhengfei,"[26, 28]",per:employee_or_member_of,People 's Liberation Army,"[36, 40]"
5,Ren Zhengfei,"[26, 28]",per:title,Deputy,"[31, 32]"
6,Ren Zhengfei,"[26, 28]",per:employee_or_member_of,Ltd.is,"[13, 14]"
7,Ltd.is,"[13, 14]",org:country_of_headquarters,Canada,"[54, 55]"
8,Huawei Technologies Co.,"[0, 3]",org:number_of_employees_members,600,"[6, 7]"
9,Huawei Technologies Co.,"[0, 3]",org:parents,Huawei Technologies Co.,"[9, 12]"
